## Loading and rendering a URDF

To run this notebook, first install trimesh, urdfpy, and pythreejs with
```
pip install trimesh urdfpy
conda install -c conda-forge pythreejs 
```

See [urdfpy documentation](https://urdfpy.readthedocs.io/en/latest/examples/index.html).

In [1]:
import math
import numpy as np

import urdfpy
import pythreejs
import gtsam
from IPython.display import display

In [142]:
robot = urdfpy.URDF.load("../../urdfs/humanoid/nao.urdf")

/home/acoadmarmon/miniconda3/lib/python3.8/site-packages/urdfpy/urdf.py:2169: RuntimeWarning: invalid value encountered in true_divide
  value = value / np.linalg.norm(value)


In [143]:
print([link.name for link in robot.links])
print([link._collision_mesh for link in robot.links])

['Neck', 'Head', 'gaze', 'LPelvis', 'LHip', 'LThigh', 'LTibia', 'LAnklePitch', 'l_ankle', 'l_sole', 'RPelvis', 'RHip', 'RThigh', 'RTibia', 'RAnklePitch', 'r_ankle', 'r_sole', 'base_link', 'torso', 'LShoulder', 'LBicep', 'LElbow', 'LForeArm', 'l_wrist', 'l_gripper', 'RShoulder', 'RBicep', 'RElbow', 'RForeArm', 'r_wrist', 'r_gripper', 'LFootBumperRight_frame', 'RFsrRL_frame', 'CameraBottom_optical_frame', 'RFsrRR_frame', 'LFsrFR_frame', 'LHandTouchLeft_frame', 'RFootBumperRight_frame', 'RFsrFR_frame', 'LFsrRR_frame', 'ImuTorsoAccelerometer_frame', 'HeadTouchFront_frame', 'RFootBumperLeft_frame', 'RSonar_frame', 'RHandTouchRight_frame', 'RHandTouchBack_frame', 'RFsrFL_frame', 'LSonar_frame', 'LFsrFL_frame', 'LFsrRL_frame', 'RHandTouchLeft_frame', 'CameraBottom_frame', 'CameraTop_frame', 'CameraTop_optical_frame', 'LHandTouchBack_frame', 'LInfraRed_frame', 'ImuTorsoGyrometer_frame', 'HeadTouchRear_frame', 'HeadTouchMiddle_frame', 'RInfraRed_frame', 'ChestButton_frame', 'LFootBumperLeft_fra

In [144]:
print(sorted({j.name:(j.parent, j.child) for j in robot.joints}.items()))

[('Accelerometer_sensor_fixedjoint', ('torso', 'ImuTorsoAccelerometer_frame')), ('CameraBottom_optical_frame_fixedjoint', ('CameraBottom_frame', 'CameraBottom_optical_frame')), ('CameraBottom_sensor_fixedjoint', ('Head', 'CameraBottom_frame')), ('CameraTop_optical_frame_fixedjoint', ('CameraTop_frame', 'CameraTop_optical_frame')), ('CameraTop_sensor_fixedjoint', ('Head', 'CameraTop_frame')), ('ChestBoard/Button_sensor_fixedjoint', ('torso', 'ChestButton_frame')), ('Gyrometer_sensor_fixedjoint', ('torso', 'ImuTorsoGyrometer_frame')), ('Head/Touch/Front_sensor_fixedjoint', ('Head', 'HeadTouchFront_frame')), ('Head/Touch/Middle_sensor_fixedjoint', ('Head', 'HeadTouchMiddle_frame')), ('Head/Touch/Rear_sensor_fixedjoint', ('Head', 'HeadTouchRear_frame')), ('HeadPitch', ('Neck', 'Head')), ('HeadYaw', ('torso', 'Neck')), ('InfraredL_sensor_fixedjoint', ('Head', 'LInfraRed_frame')), ('InfraredR_sensor_fixedjoint', ('Head', 'RInfraRed_frame')), ('LAnklePitch', ('LTibia', 'LAnklePitch')), ('LAnk

In [145]:
for joint in robot.actuated_joints:
    print(joint.name)

HeadYaw
LHipYawPitch
LShoulderPitch
RShoulderPitch
LShoulderRoll
RHipRoll
RShoulderRoll
LHipRoll
HeadPitch
RHipPitch
LHipPitch
LElbowYaw
RElbowYaw
RElbowRoll
LKneePitch
LElbowRoll
RKneePitch
RAnklePitch
RWristYaw
LWristYaw
LAnklePitch
RAnkleRoll
LAnkleRoll
LHand
RHand


In [146]:
print(robot.base_link.name)
print(robot.base_link._collision_mesh)

base_link
None


fk = robot.link_fk()
pose = fk[robot.links[4]]
print(pose)

In [148]:
visuals = robot.collision_trimesh_fk()
visuals
#robot.show()

OrderedDict([(<trimesh.Trimesh(vertices.shape=(686, 3), faces.shape=(722, 3))>,
              array([[1., 0., 0., 0.],
                     [0., 1., 0., 0.],
                     [0., 0., 1., 0.],
                     [0., 0., 0., 1.]])),
             (<trimesh.Trimesh(vertices.shape=(8, 3), faces.shape=(8, 3))>,
              array([[1.    , 0.    , 0.    , 0.    ],
                     [0.    , 1.    , 0.    , 0.    ],
                     [0.    , 0.    , 1.    , 0.1265],
                     [0.    , 0.    , 0.    , 1.    ]])),
             (<trimesh.Trimesh(vertices.shape=(4255, 3), faces.shape=(7845, 3))>,
              array([[1.    , 0.    , 0.    , 0.    ],
                     [0.    , 1.    , 0.    , 0.    ],
                     [0.    , 0.    , 1.    , 0.1265],
                     [0.    , 0.    , 0.    , 1.    ]])),
             (<trimesh.Trimesh(vertices.shape=(487, 3), faces.shape=(297, 3))>,
              array([[ 1.   ,  0.   ,  0.   ,  0.   ],
                     [

In [149]:
def buffer_geometry_from_trimesh(mesh):
  """Converts trimesh object to pythreejs.BufferGeometry."""
  # Based on tensorflow_graphics/notebooks/threejs_visualization.py
  # However, does not work!
  # This is annoyingly complex see https://threejsfundamentals.org/threejs/lessons/threejs-custom-buffergeometry.html

  geometry = pythreejs.BufferGeometry()

  #  geometry.addAttribute('position', context.THREE.BufferAttribute.new_object(vertices, 3))
  vertices = mesh.vertices.astype(np.float32)
  position = pythreejs.BufferAttribute(vertices, normalized=False)
  geometry.attributes['position'] = position

  # geometry.setIndex(context.THREE.BufferAttribute.new_object(faces, 1))
  faces = mesh.faces.astype(np.uint32).ravel()
  index = pythreejs.BufferAttribute(faces, normalized=False)
  geometry.index = index
  
  # geometry.computeVertexNormals()
  normals = mesh.vertex_normals.astype(np.float32)
  normal = pythreejs.BufferAttribute(normals, normalized=False)
  geometry.attributes['normal'] = normal

  return geometry

In [150]:
def geometry_from_trimesh(mesh):
  """Converts trimesh object to pythreejs.BufferGeometry."""

  geometry = pythreejs.Geometry()

  geometry.vertices = mesh.vertices.tolist()
  geometry.faces = [pythreejs.Face3.klass(list(f) + [None]*3) for f in mesh.faces.tolist()]
  geometry.normals = mesh.vertex_normals.tolist() # no effect?

  return geometry

In [151]:
# render last occlusion geometry for debugging
geometry = geometry_from_trimesh(list(visuals)[1])
material = pythreejs.MeshBasicMaterial(color="red")
mesh = pythreejs.Mesh( geometry, material )
mesh

Preview(child=Mesh(geometry=Geometry(colors=['#ffffff'], faces=((1, 4, 5, None, None, None), (2, 3, 7, None, N…

In [152]:
view_width = 600
view_height = 400
camera = pythreejs.CombinedCamera(position=[-1, 0, 0], width=view_width, height=view_height)

In [153]:
# Create scene
# gray = pythreejs.MeshStandardMaterial(color="gray", metalNess=0.2, roughness=0.8)
# blue = pythreejs.MeshStandardMaterial(color="blue", metalNess=0.2, roughness=0.8)
# gray = pythreejs.MeshPhysicalMaterial(color="red")
# blue = pythreejs.MeshPhysicalMaterial(color="green")
gray = pythreejs.MeshBasicMaterial(color="gray")
blue = pythreejs.MeshBasicMaterial(color="blue")
scene = pythreejs.Scene()
def xyzw(pose):
    q = pose.rotation().quaternion() # wxyz
    return q[1],q[2],q[3],q[0]
material = blue
for tm in visuals:
    matrix = visuals[tm] # this is the 4*4 transform, go figure
    pose = gtsam.Pose3(matrix) # convert to GTSAM so we can get quaternion
    geometry = geometry_from_trimesh(tm) # what we really want!
    mesh = pythreejs.Mesh(geometry, material,
                          position=pose.translation().tolist(), 
                          quaternion=xyzw(pose))
    scene.add(mesh)
    material = blue if material == gray else gray

In [154]:
key_light = pythreejs.PointLight(position=[1, 1, 1], intensity=1.5)
ambient_light = pythreejs.AmbientLight(intensity=0.4)
scene.add(key_light)
scene.add(ambient_light)
scene.add(camera)
for axis, color in [([1, 0, 0],"red"),([0, 1, 0],"green"),([0, 0, 1],"blue")]:
    gz = pythreejs.LineSegmentsGeometry(positions=[[[0, 0, 0], axis]])
    mz = pythreejs.LineMaterial(linewidth=10, color=color)
    scene.add(pythreejs.LineSegments2(gz, mz))
renderer = pythreejs.Renderer(scene=scene, camera=camera, controls=[pythreejs.OrbitControls(controlling=camera)],
                    width=view_width, height=view_height)
display(renderer)

Renderer(camera=CombinedCamera(height=400.0, position=(-1.0, 0.0, 0.0), projectionMatrix=(1.4296712803397058, …

In [13]:
from ipywidgets import interact
@interact(ortho=True)
def setOrthographic(ortho:bool):
    camera.mode = 'orthographic' if ortho else 'perspective'

interactive(children=(Checkbox(value=True, description='ortho'), Output()), _dom_classes=('widget-interact',))